In [3]:
%pylab inline
from scipy import interpolate
from scipy import optimize

Populating the interactive namespace from numpy and matplotlib


In [6]:
#U = 0 # coulob repulsion
#ne = 1.6 # number of electrons per unit cell/ occupation number
# The occupation number independent part of the Hamiltonina
nk = 200
kran = linspace(-pi,pi,nk,endpoint=False)
kx,ky = meshgrid(kran,kran)
kx = kx.flatten()
ky = ky.flatten()
gamma = -(1+exp(-1j*2*kx)+exp(-1j*(kx-ky))+exp(-1j*(kx+ky)))
H0k = zeros((len(gamma),2,2),dtype=complex)


In [3]:
# this generates eigen values and eigen vectors and returns them 
def make_spectr(U,n1,n2):
    Hk = H0k.copy()
    Hk[:,0,0] = U*n1
    Hk[:,1,1] = U*n2
    return eigh(Hk)

def FD(Energy,kbt):
    return 1/(1+exp(Energy/kbt))


In [10]:
# a function that would find the new occupation number for each site, it needs to be run in a self consistency loop, this vergion can be used with fixed point f'n     
def self_consistency2(ns,U,ne,kbt):  # (U,ns,ne)
        n1_do,n2_do,n1_up,n2_up = ns  

        vals_up,vects_up = make_spectr(U,n1_do,n2_do)
        vals_do,vects_do = make_spectr(U,n1_up,n2_up)
        num_of_k_points = len(vals_up)

        
        #########################################################################
        # get new spectrum and wavefunctions
        # vals_up[ik,i] the i-th eigenvalue of up electrons at the ik-th k point
        # vecs_up[ik,site,i] the i-th wavefunction of up electrons at the ik-th k point evaluated at site "site".
 
        
        rho,eran = histogram(array([vals_up,vals_do]).flatten(),1000,density=True)
        rho = convolve(rho,hamming(10),mode='same')/5 # some additional smoothing
        NE = cumsum(rho)*diff(eran)[0]
        mu = interpolate.interp1d(NE*4,eran[:-1],fill_value="extrapolate")
        

        
        
        n1_do_new = (sum(abs(vects_do[:, 0, 0])**2*FD(vals_do[:,0]-mu(ne),kbt))+
                    sum(abs(vects_do[:, 0, 1])**2*FD(vals_do[:,1]-mu(ne),kbt)))/num_of_k_points
                                                                                                                            #i should find fermi function of energy - mu
        n2_do_new = (sum(abs(vects_do[:, 1, 0])**2*FD(vals_do[:,0]-mu(ne),kbt))+
                    sum(abs(vects_do[:, 1, 1])**2*FD(vals_do[:,1]-mu(ne),kbt)))/num_of_k_points
        
        n1_up_new = (sum(abs(vects_up[:, 0, 0])**2*FD(vals_up[:,0]-mu(ne),kbt))+
                    sum(abs(vects_up[:, 0, 1])**2*FD(vals_up[:,1]-mu(ne),kbt)))/num_of_k_points
        
        n2_up_new =  (sum(abs(vects_up[:, 1, 0])**2*FD(vals_up[:,0]-mu(ne),kbt))+
                    sum(abs(vects_up[:, 1, 1])**2*FD(vals_up[:,1]-mu(ne),kbt)))/num_of_k_points
        
        if ((n1_up > n1_do) and (n1_up_new < n1_do_new)):
            n1_up_new = n1_do_new
        if ((n2_up > n2_do) and (n2_up_new < n2_do_new)):
            n2_up_new = n2_do_new
        if ((n1_do > n1_up) and (n1_do_new < n1_up_new)):
            n1_do_new = n1_up_new
        if ((n2_do > n2_up) and (n2_do_new < n2_up_new)):
            n2_do_new = n2_up_new
            
        
        ns_new = array([n1_do_new ,n2_do_new, n1_up_new, n2_up_new])
        ns_new = (ns_new * ne)/ sum(ns_new)

        # this is a comment
       
        return ns_new - ns

In [1]:
kbt =.01
# U = 1/.2
ne = 1.6
delta=.9
n1_up = (ne/4) + delta #occupation on site 1
n1_do = (ne/4) - delta
n2_up = (ne/4) - delta
n2_do = (ne/4) + delta
nvals_anti = []
for U in (1/arange(.1,.24,.01)):
     nvals_anti.append(optimize.root(self_consistency2,[n1_do,n2_do, n1_up,n2_up ], args=(U,ne,kbt),method='broyden2',options={'disp':True}))

In [29]:
# U=.1
# optimize.root(self_consistency2,[n1_do,n2_do, n1_up,n2_up ], args=(U,ne,kbt),method='broyden2',options={'disp':True})

In [2]:
nvals_anti

NameError: name 'nvals_anti' is not defined

In [4]:
# kbt =.01
# # U = 1/.2
# ne = 1.6
# delta=0.1
# n1_up = (ne/4) + delta #occupation on site 1
# n1_do = (ne/4) - delta
# n2_up = (ne/4) + delta
# n2_do = (ne/4) - delta
# nvals_fero = []
# for U in (1/arange(.1,.24,.01)):
#     nvals_fero.append(optimize.root(self_consistency2,[n1_do,n2_do, n1_up,n2_up ], args=(U,ne,kbt),method='broyden2',options={'disp':True}))

In [34]:
U = 1/.13
n1_do,n2_do, n1_up,n2_up = 0.09010787, 0.70989213, 0.70989213, 0.09010787
E_up = U * ((n1_up+n2_up)/2) - U * ((n1_up*n1_do)+(n2_up*n2_do)) + 0.5 * sqrt(U**2 * (n1_up - n2_up)**2 + 4**3)
E_do = U * ((n1_do+n2_do)/2) - U * ((n1_up*n1_do)+(n2_up*n2_do)) + 0.5 * sqrt(U**2 * (n1_do - n2_do)**2 + 4**3)

In [35]:
E_up

4.035571479228923